In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata
import tqdm
from time import sleep
import time

In [3]:
df = pd.read_hdf("df_prep.h5", key="df")
df

,idn,title,date,lang,urn,title_clean
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...
...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren


# Vorbereitungen:

## Laden spezifischer Bibliotheken: 

In [4]:
import spacy
from spacy import displacy
import os
import plotly.graph_objects as go
import plotly.express as px

In [5]:
#Code für den Download eines der Sprachmodelle: 
#!python -m spacy download de

In [6]:
nlp = spacy.load('de_core_news_sm') ## Laden eines Trained Model for German

In [7]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']


## Tokenization: 

In [8]:
def process_text(text):
    return nlp(text)

In [9]:
df['token'] = df['title_clean'].apply(process_text)
df

,idn,title,date,lang,urn,title_clean,token
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor..."
...,...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie,"(zwei, nachbarländer, pandemie)"
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen,"(zwei, runden, gehen, corona, urologie, ende, ..."
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...,"(arbeitserleichterung, de-professionalisierung..."
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren,"(pandemie, wahlkampf, verhältnismäßigkeit, wah..."


## Lemmatization: 

In [10]:
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

In [11]:
df['lemma'] = df['token'].apply(get_lemma)
df

,idn,title,date,lang,urn,title_clean,token,lemma
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ...","[2020, Jahr, Welt, verändert, amerikanisch, So..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb...","[Aufbau, key, Account, Management, Tourismusbr..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge...","[auswirkung, corona-pandemie, Ausbildung, gesu..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu..."
...,...,...,...,...,...,...,...,...
11363,129882589X,"zwei nachbarländer, eine pandemie",2023,ger,urn:nbn:de:101:1-2023080821170040558967,zwei nachbarländer pandemie,"(zwei, nachbarländer, pandemie)","[zwei, nachbarländ, Pandemie]"
11364,1232892661,zwei runden gehen an corona – aber die urologi...,2021,ger,urn:nbn:de:101:1-2021050520504432857162,zwei runden gehen corona urologie ende gewinnen,"(zwei, runden, gehen, corona, urologie, ende, ...","[zwei, Runde, gehen, Corona, Urologie, Ende, g..."
11365,1307057543,zwischen arbeitserleichterung und de-professio...,2023,ger,urn:nbn:de:101:1-2023102409011131884864,arbeitserleichterung de-professionalisierungsg...,"(arbeitserleichterung, de-professionalisierung...","[Arbeitserleichterung, de-professionalisierung..."
11366,1241532591,zwischen pandemie und wahlkampf: verhältnismäß...,2021,ger,urn:nbn:de:101:1-2021092021390437649681,pandemie wahlkampf verhältnismäßigkeit wahren,"(pandemie, wahlkampf, verhältnismäßigkeit, wah...","[Pandemie, Wahlkampf, Verhältnismäßigkeit, wahr]"


### Einfache Worthäufigkeitsprüfung: 

In [11]:
# Hier zu suchenden Begriff eingeben: 
term = "corona"  

In [12]:
print(f'"{term}" appears in the text tokens column ' + str(df.token.apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')#
print(f'"{term}" appears in the lemmas column ' + str(df['lemma'].apply(lambda x: str(x)).apply(lambda x: x.count(term)).sum()) + ' times.')

"corona" appears in the text tokens column 2597 times.
"corona" appears in the lemmas column 1900 times.


## Optional: Part of speech tags

In [13]:
def get_pos(doc):
    return [(token.pos_, token.tag_) for token in doc]

In [14]:
df['POS'] = df['token'].apply(get_pos)
df.head()

,idn,title,date,lang,urn,title_clean,token,lemma,POS
0,1285990005,"2020 das jahr, das die welt veränderte der ame...",2024,ger,N/A,2020 jahr welt veränderte amerikanische soziol...,"(2020, jahr, welt, veränderte, amerikanische, ...","[2020, Jahr, Welt, verändert, amerikanisch, So...","[(NUM, CARD), (NOUN, NN), (NOUN, NN), (ADJ, AD..."
1,1317002237,ängste der allgemeinbevölkerung in zeiten der ...,2024,ger,urn:nbn:de:bsz:14-qucosa2-890970,ängste allgemeinbevölkerung zeiten covid-19-pa...,"(ängste, allgemeinbevölkerung, zeiten, covid-1...","[Angst, Allgemeinbevölkerung, Zeit, covid-19-p...","[(ADJ, ADJA), (NOUN, NN), (NOUN, NN), (PROPN, ..."
2,1319076106,aufbau eines key account managements in der to...,2024,ger,urn:nbn:de:101:1-2024021302301350289147,aufbau key account managements tourismusbranch...,"(aufbau, key, account, managements, tourismusb...","[Aufbau, key, Account, Management, Tourismusbr...","[(NOUN, NN), (ADP, APPR), (PROPN, NE), (PROPN,..."
4,1315575361,auswirkungen der corona-pandemie auf die ausbi...,2024,ger,urn:nbn:de:101:1-2024011102383134159187,auswirkungen corona-pandemie ausbildung gesund...,"(auswirkungen, corona-pandemie, ausbildung, ge...","[auswirkung, corona-pandemie, Ausbildung, gesu...","[(ADJ, ADJA), (NOUN, NN), (NOUN, NN), (VERB, V..."
5,131701071X,auswirkungen der covid-19-pandemie auf die pri...,2024,ger,urn:nbn:de:bsz:16-heidok-342525,auswirkungen covid-19-pandemie primärversorgun...,"(auswirkungen, covid-19-pandemie, primärversor...","[auswirkung, covid-19-pandemie, Primärversorgu...","[(ADJ, ADJA), (ADJ, FM), (NOUN, NN), (ADJ, ADJ..."


# Topic Modeling:

### Latent Dirichlet Allocation (LDA):¶

In [12]:
import gensim
from gensim import corpora

In [13]:
lemmatized_text = df.token.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))


In [14]:
# Create a dictionary and document-term matrix for topic modeling
dictionary = corpora.Dictionary(lemmatized_text.apply(lambda x: x.split()))
doc_term_matrix = [dictionary.doc2bow(doc.split()) for doc in lemmatized_text]

In [15]:
# Apply LDA for topic modeling
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=3, id2word=dictionary, passes=15)

In [16]:
# Visualize the topics
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.182685  0.024657       1        1  38.509215
2      0.067635 -0.154834       2        1  37.858066
0      0.115050  0.130177       3        1  23.632719, topic_info=                  Term         Freq        Total Category  logprob  loglift
84                  --  2312.000000  2312.000000  Default  30.0000  30.0000
18              Corona   310.000000   310.000000  Default  29.0000  29.0000
11                Zeit   481.000000   481.000000  Default  28.0000  28.0000
4             Pandemie  1176.000000  1176.000000  Default  27.0000  27.0000
13   covid-19-pandemie   550.000000   550.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
169        Deutschland    42.160606   200.625772   Topic3  -5.7926  -0.1174
143         sars-cov-2    32.616067    87.663468   Topic3  -6.0493   0.4538
128             sozial    35.023396   164.690339   Topic3  -5.9780  -0.1055
53     Herausforderung    31.769960   111.284277   Topic3  -6.0755   0.1890
96        corona-krise    30.963065   112.896873   Topic3  -6.1013   0.1489

[192 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
101       3  0.979149           -
84        1  0.999772          --
1160      1  0.995362           1
1084      2  0.986170          12
4228      3  0.991923          14
...     ...       ...         ...
91        1  0.098578       die
91        2  0.679095       die
91        3  0.221801       die
689       1  0.998157           §
317       3  0.987090  öffentlich

[217 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])